# Distributed TensorFlow with Horovod
In this tutorial, you will train a word2vec model in TensorFlow using distributed training via [Horovod](https://github.com/uber/horovod).

> Horovod is a distributed training framework for TensorFlow, Keras, PyTorch, and MXNet. The goal of Horovod is to make distributed Deep Learning fast and easy to use.

<img src="https://docs.microsoft.com/en-us/azure/machine-learning/service/media/overview-what-is-azure-ml/aml.png">

The TensorFlow estimator also supports distributed training across CPU and GPU clusters. You can easily run distributed TensorFlow jobs and Azure Machine Learning will manage the orchestration for you.

Azure Machine Learning supports two methods of distributed training in TensorFlow:

1. **MPI-based distributed training** using the **Horovod** framework
2. **Native distributed TensorFlow** using the parameter server method

**GPU optimized VM sizes are specialized virtual machines** available with single or multiple NVIDIA GPUs. These sizes are designed for compute-intensive, graphics-intensive, and visualization workloads. This article provides information about the number and type of GPUs, vCPUs, data disks, and NICs. Storage throughput and network bandwidth are also included for each size in this grouping.

- **NC, NCv2, NCv3** sizes are optimized for compute-intensive and network-intensive applications and algorithms. Some examples are CUDA- and OpenCL-based applications and simulations, AI, and Deep Learning. The NCv3-series is focused on high-performance computing workloads featuring NVIDIA’s Tesla V100 GPU. The NC-series uses the Intel Xeon E5-2690 v3 2.60GHz v3 (Haswell) processor, and the NCv2-series and NCv3-series VMs use the Intel Xeon E5-2690 v4 (Broadwell) processor.

- **ND, and NDv2** The ND-series is focused on training and inference scenarios for deep learning. It uses the NVIDIA Tesla P40 GPU and the Intel Xeon E5-2690 v4 (Broadwell) processor. The NDv2-series uses the Intel Xeon Platinum 8168 (Skylake) processor.

- **NV and NVv3** sizes are optimized and designed for remote visualization, streaming, gaming, encoding, and VDI scenarios using frameworks such as OpenGL and DirectX. These VMs are backed by the NVIDIA Tesla M60 GPU.

<img src="https://docs.microsoft.com/en-us/azure/architecture/reference-architectures/ai/_images/distributed_dl_flow.png">

## 1. Infos

In [62]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [63]:
import datetime
now = datetime.datetime.now()
print(now)

2019-11-13 09:13:31.348033


In [64]:
# Check core SDK version number
import azureml.core

print("Azure ML service version = ", azureml.core.VERSION)

Azure ML service version =  1.0.72


## 2. Connexion Azure ML service

In [65]:
#Workspace
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "70b8f39e-8863-49f7-b6ba-34a80799550c")
resource_group = os.environ.get("RESOURCE_GROUP", "azuremlserviceresourcegroup")
workspace_name = os.environ.get("WORKSPACE_NAME", "azuremlservice")


from azureml.core import Workspace
try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("OK")
except:
   print("Error: Workspace not found")

OK


In [66]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: azuremlservice
Azure region: westeurope
Resource group: azuremlserviceresourcegroup


## 3. Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

**Standard_NC6 :
6 vCPU - 56 Gb Ram - 340 GB SSD - 1 GPU- 12 Gb GPU Memory**

> https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-gpu<br>

In [67]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-11-13T07:59:19.867000+00:00', 'errors': None, 'creationTime': '2019-11-12T15:26:15.321861+00:00', 'modifiedTime': '2019-11-12T15:26:31.231689+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


The above code creates a GPU cluster. If you instead want to create a CPU cluster, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## 4. Create a Dataset for Files
A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. FileDataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [68]:
from azureml.core import Dataset

web_paths = ['http://mattmahoney.net/dc/text8.zip']
dataset = Dataset.File.from_files(path=web_paths)

You may want to register datasets using the register() method to your workspace so that the dataset can be shared with others, reused across various experiments, and referred to by name in your training script.

In [69]:
dataset = dataset.register(workspace=ws,
                           name='mattmahoney dataset',
                           description='mattmahoney training and test dataset',
                           create_new_version=True)

In [70]:
# list the files referenced by the dataset
dataset.to_path()

array(['/text8.zip'], dtype=object)

## 5. Train model on the remote compute

### 5.1 Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [71]:
import os
project_folder = './tf-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_horovod_word2vec.py` into this project directory.

In [72]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

'./tf-distr-hvd/tf_horovod_word2vec.py'

### 5.2 Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [73]:
from azureml.core import Experiment

experiment_name = 'TF-horovod'
experiment = Experiment(ws, name=experiment_name)

### 5.3 Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

Horovod is an open-source framework for distributed training developed by Uber. It offers an easy path to distributed GPU TensorFlow jobs.

To use Horovod, specify an MpiConfiguration object for the distributed_training parameter in the TensorFlow constructor. This parameter ensures that Horovod library is installed for you to use in your training script.

In [74]:
from azureml.train.dnn import TensorFlow, Mpi

script_params={
    '--input_data': dataset.as_named_input('mattmahoney').as_mount(),
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      distributed_training=Mpi(),
                      framework_version='1.13', 
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend=Mpi()`. To specify `i` workers per node, you must provide the argument `distributed_backend=Mpi(process_count_per_node=i)`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

### 5.4 Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [75]:
run = experiment.submit(estimator)
print(run)
run.get_details()

Run(Experiment: TF-horovod,
Id: TF-horovod_1573636453_7e86d133,
Type: azureml.scriptrun,
Status: Starting)


{'runId': 'TF-horovod_1573636453_7e86d133',
 'target': 'gpu-cluster',
 'status': 'Queued',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': 'a61d8890-9078-4e00-913d-4b143e7b2b58',
  'AzureML.DerivedImageName': 'azureml/azureml_75651667c3311d0b90ca821953173db3',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '113c9925-6a23-467e-8106-8a7c0ce308a4'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mattmahoney', 'mechanism': 'Mount'}}],
 'runDefinition': {'script': 'tf_horovod_word2vec.py',
  'arguments': ['--input_data', 'DatasetConsumptionConfig:mattmahoney'],
  'sourceDirectoryDataStore': 'workspaceblobstore',
  'framework': 'Python',
  'communicator': 'Mpi',
  'target': 'gpu-cluster',
  'dataReferences': {'workspaceblobstore': {'dataStoreName': 'workspaceblobstore',
    'mode': 'Mount',
    'pathOnDataStore': None,
    'pathOnCompute': None,


### 5.5 Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [76]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [79]:
run.get_details()

{'runId': 'TF-horovod_1573636453_7e86d133',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2019-11-13T09:18:26.141652Z',
 'endTimeUtc': '2019-11-13T09:23:39.968915Z',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': 'a61d8890-9078-4e00-913d-4b143e7b2b58',
  'AzureML.DerivedImageName': 'azureml/azureml_75651667c3311d0b90ca821953173db3',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '113c9925-6a23-467e-8106-8a7c0ce308a4'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mattmahoney', 'mechanism': 'Mount'}}],
 'runDefinition': {'script': 'tf_horovod_word2vec.py',
  'arguments': ['--input_data', 'DatasetConsumptionConfig:mattmahoney'],
  'sourceDirectoryDataStore': 'workspaceblobstore',
  'framework': 'Python',
  'communicator': 'Mpi',
  'target': 'gpu-cluster',
  'dataReferences': {'workspaceblobstore': {'dataStoreName': 'w

In [43]:
#run.wait_for_completion(show_output=True)